# DEEP NEURAL NETWORKS - ASSIGNMENT 2: CNN FOR IMAGE CLASSIFICATION

## Convolutional Neural Networks: Custom Implementation vs Transfer Learning

STUDENT INFORMATION (REQUIRED - DO NOT DELETE)

BITS ID: [Enter your BITS ID here - e.g., 2025AA1234]

Name: [Enter your full name here - e.g., JOHN DOE]

Email: [Enter your email]

Date: [Submission date]

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import time
import json
import os
import random
from PIL import Image, UnidentifiedImageError # Added UnidentifiedImageError for robust cleaning
import shutil # For copying files

# Deep learning frameworks
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, VGG16

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Configure TensorFlow to dynamically allocate GPU memory
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs configured for memory growth.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected by TensorFlow. Running on CPU.")


"""
PART 1: DATASET LOADING AND EXPLORATION
"""

# --- Data Preparation: Assume Pre-Organized Data ---
# This section assumes your data is already in the 'PetImages/Train/Cat', 'PetImages/Train/Dog',
# 'PetImages/Test/Cat', 'PetImages/Test/Dog' structure relative to your notebook.

# Step 0: Initial Cleanup of any previous *cleaned* data directories
print("\n--- Step 0: Initial Cleanup of (previous) cleaned data ---")
!rm -rf Cleaned_PetImages_For_Training # Remove any pre-existing cleaned data directory from a previous run
print("Cleanup complete.")

# Define paths to your pre-organized data
SOURCE_DATA_BASE_DIR = 'PetImages'
raw_train_dir = os.path.join(SOURCE_DATA_BASE_DIR, 'Train')
raw_test_dir = os.path.join(SOURCE_DATA_BASE_DIR, 'Test')

# Verify that the source directories exist
if not os.path.exists(raw_train_dir):
    raise FileNotFoundError(f"Source Training directory not found: {raw_train_dir}. Please create it and place images.")
if not os.path.exists(raw_test_dir):
    raise FileNotFoundError(f"Source Testing directory not found: {raw_test_dir}. Please create it and place images.")

print(f"\n--- Detected pre-organized data in '{SOURCE_DATA_BASE_DIR}' ---")


# Step 1: Set up NEW, CLEAN directories for training (to copy only verified good images)
CLEANED_DATASET_BASE = 'Cleaned_PetImages_For_Training' # New folder for processed data
cleaned_train_dir = os.path.join(CLEANED_DATASET_BASE, 'Train')
cleaned_test_dir = os.path.join(CLEANED_DATASET_BASE, 'Test')

os.makedirs(os.path.join(cleaned_train_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(cleaned_train_dir, 'Dog'), exist_ok=True)
os.makedirs(os.path.join(cleaned_test_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(cleaned_test_dir, 'Dog'), exist_ok=True)
print(f"Created new directories for cleaned data at: {CLEANED_DATASET_BASE}")


# Step 2: Meticulous Cleaning and Copying to New Directories (No Deletion of originals)
print("\n--- Step 2: Verifying and Copying Good Images to Cleaned Directories ---")

good_image_paths_by_category = {'Cat': [], 'Dog': []}
skipped_file_count = 0
verified_file_count = 0

for source_dir in [raw_train_dir, raw_test_dir]:
    for category in ['Cat', 'Dog']:
        category_path = os.path.join(source_dir, category)
        if not os.path.exists(category_path):
            print(f"Warning: Category path not found: {category_path}. Skipping.")
            continue
            
        print(f"Scanning {category_path}...")
        for img_name in os.listdir(category_path):
            img_path_src = os.path.join(category_path, img_name)
            if not os.path.isfile(img_path_src): continue # Skip non-file entries

            try:
                # Check for zero-byte files
                if os.path.getsize(img_path_src) == 0:
                    skipped_file_count += 1
                    continue
                
                with Image.open(img_path_src) as img:
                    img.verify() # Verify file integrity
                    img.convert('RGB') # Forces full loading and conversion
                
                # If all steps above succeed, the image is considered good
                good_image_paths_by_category[category].append(img_path_src)
                verified_file_count += 1

            except (IOError, SyntaxError, UnidentifiedImageError, AttributeError) as e:
                skipped_file_count += 1
                # print(f"  --> Skipping corrupted/unidentifiable image: {img_path_src} - Error: {type(e).__name__}: {e}") # Uncomment for debugging
                pass # Do not delete, just skip
    
    if (verified_file_count + skipped_file_count) % 1000 == 0 and (verified_file_count + skipped_file_count) > 0: # Progress indicator
        print(f"Processed {verified_file_count + skipped_file_count} images. Verified: {verified_file_count}, Skipped: {skipped_file_count}.")

print(f"\n--- Finished checking all raw images. ---")
print(f"Total VERIFIED images: {verified_file_count}")
print(f"Total SKIPPED images (zero-byte/corrupted): {skipped_file_count}")
print(f"Verified good Cat images found: {len(good_image_paths_by_category['Cat'])}")
print(f"Verified good Dog images found: {len(good_image_paths_by_category['Dog'])}")


# Step 3: Split and Copy ONLY the good images to the new structure
final_train_samples = 0
final_test_samples = 0
split_ratio = 0.9 # This split is applied to the combined good images

print("\n--- Step 3: Copying good images to final Train/Test directories ---")
for category, img_list in good_image_paths_by_category.items():
    random.shuffle(img_list) # Shuffle good images before splitting
    split_idx = int(len(img_list) * split_ratio)
    
    train_images = img_list[:split_idx]
    test_images = img_list[split_idx:]
    
    # Copy to cleaned_train_dir
    print(f"Copying {len(train_images)} train images for {category}...")
    for img_path_src in train_images:
        img_name = os.path.basename(img_path_src)
        shutil.copy(img_path_src, os.path.join(cleaned_train_dir, category, img_name))
    final_train_samples += len(train_images)

    # Copy to cleaned_test_dir
    print(f"Copying {len(test_images)} test images for {category}...")
    for img_path_src in test_images:
        img_name = os.path.basename(img_path_src)
        shutil.copy(img_path_src, os.path.join(cleaned_test_dir, category, img_name))
    final_test_samples += len(test_images)

print(f"\nFinal total training samples: {final_train_samples}")
print(f"Final total testing samples: {final_test_samples}")
print("--- Cleaning and Copying Complete ---")


# --- Dataset Metadata (using the final, cleaned counts) ---
dataset_name = "Cats vs Dogs (User-provided, Cleaned & Structured)"
dataset_source = "User-provided local data"
n_samples = final_train_samples + final_test_samples
n_classes = 2 # Fixed for Cats vs Dogs

n_cat_final_train = len(os.listdir(os.path.join(cleaned_train_dir, 'Cat')))
n_dog_final_train = len(os.listdir(os.path.join(cleaned_train_dir, 'Dog')))
n_cat_final_test = len(os.listdir(os.path.join(cleaned_test_dir, 'Cat')))
n_dog_final_test = len(os.listdir(os.path.join(cleaned_test_dir, 'Dog')))
samples_per_class = f"Train (Cat): {n_cat_final_train}, (Dog): {n_dog_final_train} | Test (Cat): {n_cat_final_test}, (Dog): {n_dog_final_test}"

# Image shape (adjusted for memory management)
image_shape = [150, 150, 3] # Using 150x150 as a balance. Can reduce to 64x64 if OOM occurs.
problem_type = "classification"
primary_metric = "accuracy"
metric_justification = """
Accuracy is chosen as the primary metric because the Cats vs Dogs dataset is relatively balanced.
In a balanced dataset, accuracy provides a good overall measure of the model's performance,
indicating the proportion of correctly classified images among all images.
"""

print("\n--- Updated DATASET INFORMATION ---")
print(f"Dataset: {dataset_name}")
print(f"Source: {dataset_source}")
print(f"Total Samples: {n_samples}")
print(f"Number of Classes: {n_classes}")
print(f"Samples per Class: {samples_per_class}")
print(f"Image Shape: {image_shape}")
print(f"Primary Metric: {primary_metric}")
print(f"Metric Justification: {metric_justification}")

# Required: Document your split
# Added a check to prevent ZeroDivisionError if n_samples is 0
if n_samples > 0:
    train_test_ratio = f"{round(final_train_samples/n_samples*100)}/{round(final_test_samples/n_samples*100)}"
else:
    train_test_ratio = "0/0" # No samples, so ratio is 0/0

print(f"\nTrain/Test Split: {train_test_ratio}")
print(f"Training Samples: {final_train_samples}")
print(f"Test Samples: {final_test_samples}")

### 1.2 Data Exploration and Visualization

In [ ]:
# --- ImageDataGenerator Setup ---
IMG_HEIGHT, IMG_WIDTH = image_shape[0], image_shape[1]
BATCH_SIZE = 8 # Adjusted for memory management. Can try 4 if 8 fails.

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# IMPORTANT: Point generators to the CLEANED_DATASET_BASE directories
train_generator = train_datagen.flow_from_directory(
    cleaned_train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    cleaned_test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=SEED,
    shuffle=False
)

# --- Class Distribution Plotting ---
class_labels = list(train_generator.class_indices.keys())

train_counts_plot = {label: 0 for label in class_labels}
num_batches_train_plot = len(train_generator)
if num_batches_train_plot == 0:
    print("Warning: train_generator is empty for plotting. No training data.")
else:
    for i in range(num_batches_train_plot):
        try:
            _, labels = train_generator[i]
            for label_one_hot in labels:
                class_index = np.argmax(label_one_hot)
                if class_index < len(class_labels):
                    train_counts_plot[class_labels[class_index]] += 1
        except Exception as e:
            print(f"Error during train_generator iteration for plotting at batch {i}: {e}")
            break
train_counts_plot = {k:v for k,v in train_counts_plot.items() if v>0}

test_counts_plot = {label: 0 for label in class_labels}
num_batches_test_plot = len(test_generator)
if num_batches_test_plot == 0:
    print("Warning: test_generator is empty for plotting. No test data.")
else:
    for i in range(num_batches_test_plot):
        try:
            _, labels = test_generator[i]
            for label_one_hot in labels:
                class_index = np.argmax(label_one_hot)
                if class_index < len(class_labels):
                    test_counts_plot[class_labels[class_index]] += 1
        except Exception as e:
            print(f"Error during test_generator iteration for plotting at batch {i}: {e}")
            break
test_counts_plot = {k:v for k,v in test_counts_plot.items() if v>0}


if not train_counts_plot:
    print("Warning: No data to plot for Training Class Distribution.")
else:
    plt.figure(figsize=(10, 5))
    sns.barplot(x=list(train_counts_plot.keys()), y=list(train_counts_plot.values()))
    plt.title('Training Class Distribution')
    plt.xlabel('Class')
    plt.ylabel('Count')
    plt.show()

if not test_counts_plot:
    print("Warning: No data to plot for Test Class Distribution.")
else:
    plt.figure(figsize=(10, 5))
    sns.barplot(x=list(test_counts_plot.keys()), y=list(test_counts_plot.values()))
    plt.title('Test Class Distribution')
    plt.xlabel('Class')
    plt.ylabel('Count')
    plt.show()

### 2.1 Custom CNN Architecture Design

In [ ]:
def build_custom_cnn(input_shape, n_classes):
    """
    Build custom CNN architecture
    """
    inputs = Input(shape=input_shape)
    
    # First convolutional block
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.25)(x)
    
    # Second convolutional block
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.25)(x)

    # Third convolutional block
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.25)(x)
    
    # Global Average Pooling (MANDATORY)
    x = GlobalAveragePooling2D()(x)
    
    # Dense layers for classification head
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(n_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    return model

# Create model instance
custom_cnn = build_custom_cnn(image_shape, n_classes)

# Compile model
custom_cnn.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

custom_cnn.summary()

### 2.2 Train Custom CNN

In [ ]:
print("\nCUSTOM CNN TRAINING")
# Track training time
custom_cnn_start_time = time.time()

EPOCHS = 20 # Can be adjusted

# History object to store training metrics
history_custom_cnn = custom_cnn.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=test_generator.samples // BATCH_SIZE,
    verbose=1
)

custom_cnn_training_time = time.time() - custom_cnn_start_time

# REQUIRED: Track initial and final loss
custom_cnn_initial_loss = float(history_custom_cnn.history['loss'][0])
custom_cnn_final_loss = float(history_custom_cnn.history['loss'][-1])

print(f"Training completed in {custom_cnn_training_time:.2f} seconds")
print(f"Initial Loss: {custom_cnn_initial_loss:.4f}")
print(f"Final Loss: {custom_cnn_final_loss:.4f}")

### 2.3 Evaluate Custom CNN

In [ ]:
print("\nCUSTOM CNN EVALUATION")

# Get predictions
y_pred_probs_cnn = custom_cnn.predict(test_generator, steps=(test_generator.samples // BATCH_SIZE) + 1)
y_pred_cnn = np.argmax(y_pred_probs_cnn, axis=1)

y_true_cnn = test_generator.classes[:len(y_pred_cnn)]


# Calculate all 4 metrics
custom_cnn_accuracy = float(accuracy_score(y_true_cnn, y_pred_cnn))
custom_cnn_precision = float(precision_score(y_true_cnn, y_pred_cnn, average='macro', zero_division=0))
custom_cnn_recall = float(recall_score(y_true_cnn, y_pred_cnn, average='macro', zero_division=0))
custom_cnn_f1 = float(f1_score(y_true_cnn, y_pred_cnn, average='macro', zero_division=0))

print("\nCustom CNN Performance:")
print(f"Accuracy:  {custom_cnn_accuracy:.4f}")
print(f"Precision: {custom_cnn_precision:.4f}")
print(f"Recall:    {custom_cnn_recall:.4f}")
print(f"F1-Score:  {custom_cnn_f1:.4f}")

### 2.4 Visualize Custom CNN Results

In [ ]:
# Plot training loss curve
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_custom_cnn.history['loss'], label='Train Loss')
plt.plot(history_custom_cnn.history['val_loss'], label='Validation Loss')
plt.title('Custom CNN Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_custom_cnn.history['accuracy'], label='Train Accuracy')
plt.plot(history_custom_cnn.history['val_accuracy'], label='Validation Accuracy')
plt.title('Custom CNN Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot confusion matrix
cm_cnn = confusion_matrix(y_true_cnn, y_pred_cnn)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_cnn, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Custom CNN Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Show sample predictions
def plot_sample_predictions(generator, model, class_labels, num_samples=5):
    # Get one batch of images and labels
    x, y_true_one_hot = next(generator)
    
    # Adapt samples_to_plot to the actual size of the batch received
    actual_batch_size = x.shape[0]
    samples_to_plot = min(num_samples, actual_batch_size)
    
    y_true = np.argmax(y_true_one_hot, axis=1)
    y_pred_probs = model.predict(x)
    y_pred = np.argmax(y_pred_probs, axis=1)

    plt.figure(figsize=(15, 6))
    for i in range(samples_to_plot): # Loop only for available samples
        plt.subplot(1, samples_to_plot, i + 1) # Adjust subplot grid to actual samples shown
        plt.imshow(x[i])
        
        true_label_text = class_labels[y_true[i]] if y_true[i] < len(class_labels) else f"Unknown ({y_true[i]})"
        pred_label_text = class_labels[y_pred[i]] if y_pred[i] < len(class_labels) else f"Unknown ({y_pred[i]})"

        plt.title(f"True: {true_label_text}\nPred: {pred_label_text}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

print("\nSample Custom CNN Predictions:")
plot_sample_predictions(test_generator, custom_cnn, class_labels)

### 3.1 Load Pre-trained Model and Modify Architecture

In [ ]:
print("\n" + "="*70)
print("TRANSFER LEARNING IMPLEMENTATION")

pretrained_model_name = "ResNet50" # Chosen pre-trained model

def build_transfer_learning_model(base_model_name, input_shape, n_classes):
    """
    Build transfer learning model
    """
    # Load pre-trained model without top layers
    if base_model_name == "ResNet50":
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == "VGG16":
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Invalid base_model_name. Choose ResNet50 or VGG16.")
        
    # Freeze base layers
    for layer in base_model.layers:
        layer.trainable = False
        
    # Add Global Average Pooling + custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x) # MANDATORY GAP
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(n_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=outputs)
    
    # Compile model (lower learning rate for fine-tuning)
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Create transfer learning model
transfer_model = build_transfer_learning_model(pretrained_model_name, image_shape, n_classes)

# Count layers and parameters
frozen_layers = len([layer for layer in transfer_model.layers if not layer.trainable])
trainable_layers = len([layer for layer in transfer_model.layers if layer.trainable])
total_parameters = transfer_model.count_params()

# Corrected trainable_parameters calculation using tf.size()
trainable_parameters = sum(tf.size(variable).numpy() for variable in transfer_model.trainable_weights)


print(f"Base Model: {pretrained_model_name}")
print(f"Frozen Layers: {frozen_layers}")
print(f"Trainable Layers: {trainable_layers}")
print(f"Total Parameters: {total_parameters:,}")
print(f"Trainable Parameters: {trainable_parameters:,}")
print(f"Using Global Average Pooling: YES")

transfer_model.summary()

### 3.2 Train Transfer Learning Model

In [ ]:
print("\nTraining Transfer Learning Model...")

tl_learning_rate = 0.0001
tl_epochs = 15 # Fine-tuning usually needs fewer epochs
tl_batch_size = BATCH_SIZE # Use the same batch size as custom CNN for consistency
tl_optimizer = "Adam"

tl_start_time = time.time()

history_transfer_learning = transfer_model.fit(
    train_generator,
    epochs=tl_epochs,
    validation_data=test_generator,
    steps_per_epoch=train_generator.samples // tl_batch_size,
    validation_steps=test_generator.samples // tl_batch_size,
    verbose=1
)

tl_training_time = time.time() - tl_start_time

tl_initial_loss = float(history_transfer_learning.history['loss'][0])
tl_final_loss = float(history_transfer_learning.history['loss'][-1])

print(f"Training completed in {tl_training_time:.2f} seconds")
print(f"Initial Loss: {tl_initial_loss:.4f}")
print(f"Final Loss: {tl_final_loss:.4f}")

### 3.3 Evaluate Transfer Learning Model

In [ ]:
print("\nTransfer Learning EVALUATION")

y_pred_probs_tl = transfer_model.predict(test_generator, steps=(test_generator.samples // tl_batch_size) + 1)
y_pred_tl = np.argmax(y_pred_probs_tl, axis=1);

y_true_tl = test_generator.classes[:len(y_pred_tl)];


tl_accuracy = float(accuracy_score(y_true_tl, y_pred_tl));
tl_precision = float(precision_score(y_true_tl, y_pred_tl, average='macro', zero_division=0));
tl_recall = float(recall_score(y_true_tl, y_pred_tl, average='macro', zero_division=0));
tl_f1 = float(f1_score(y_true_tl, y_pred_tl, average='macro', zero_division=0));

print("\nTransfer Learning Performance:")
print(f"Accuracy:  {tl_accuracy:.4f}")
print(f"Precision: {tl_precision:.4f}")
print(f"Recall:    {tl_recall:.4f}")
print(f"F1-Score:  {tl_f1:.4f}")

### 3.4 Visualize Transfer Learning Results

In [ ]:
# Plot training loss curve
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_transfer_learning.history['loss'], label='Train Loss')
plt.plot(history_transfer_learning.history['val_loss'], label='Validation Loss')
plt.title('Transfer Learning Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_transfer_learning.history['accuracy'], label='Train Accuracy')
plt.plot(history_transfer_learning.history['val_accuracy'], label='Validation Accuracy')
plt.title('Transfer Learning Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot confusion matrix
cm_tl = confusion_matrix(y_true_tl, y_pred_tl)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_tl, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Transfer Learning Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

print("\nSample Transfer Learning Predictions:")
plot_sample_predictions(test_generator, transfer_model, class_labels)

### 4.1 Metrics Comparison

In [ ]:
print("\n" + "="*70)
print("MODEL COMPARISON")

comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Training Time (s)', 'Total Parameters', 'Trainable Parameters'],
    'Custom CNN': [
        custom_cnn_accuracy,
        custom_cnn_precision,
        custom_cnn_recall,
        custom_cnn_f1,
        custom_cnn_training_time,
        float(custom_cnn.count_params()),
        float(custom_cnn.count_params()) # All parameters are trainable for custom CNN
    ],
    'Transfer Learning': [
        tl_accuracy,
        tl_precision,
        tl_recall,
        tl_f1,
        tl_training_time,
        float(total_parameters),
        float(trainable_parameters)
    ]
})

print(comparison_df.to_string(index=False))

### 4.2 Visual Comparison

In [ ]:
# Visual Comparison
# Bar plot comparing key metrics
metrics_to_compare = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
bar_data = comparison_df[comparison_df['Metric'].isin(metrics_to_compare)].set_index('Metric')

bar_data.plot(kind='bar', figsize=(10, 6))
plt.title('Model Performance Comparison (Metrics)')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Plot training curves comparison (Loss)
plt.figure(figsize=(12, 5))
plt.plot(history_custom_cnn.history['val_loss'], label='Custom CNN Val Loss', linestyle='--')
plt.plot(history_transfer_learning.history['val_loss'], label='Transfer Learning Val Loss')
plt.title('Validation Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Plot training curves comparison (Accuracy)
plt.figure(figsize=(12, 5))
plt.plot(history_custom_cnn.history['val_accuracy'], label='Custom CNN Val Accuracy', linestyle='--')
plt.plot(history_transfer_learning.history['val_accuracy'], label='Transfer Learning Val Accuracy')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

### 4.2 Visual Comparison

In [ ]:
analysis_text = f"""
The Transfer Learning (TL) model (ResNet50) significantly outperformed the Custom CNN. 
TL achieved an accuracy of {tl_accuracy:.4f}, precision of {tl_precision:.4f}, recall of {tl_recall:.4f}, and F1-score of {tl_f1:.4f}, 
whereas the Custom CNN yielded {custom_cnn_accuracy:.4f} accuracy, {custom_cnn_precision:.4f} precision, {custom_cnn_recall:.4f} recall, and {custom_cnn_f1:.4f} F1-score. 
This superior performance by TL highlights the immense impact of pre-training on large datasets like ImageNet, 
allowing the model to leverage learned hierarchical features from a broad domain. 
Training from scratch with limited data makes it challenging for a custom model to learn robust, generalizable features.

Global Average Pooling (GAP) was crucial for both models. 
It effectively reduced the number of parameters in the classification head, thereby mitigating overfitting 
and acting as a structural regularizer compared to traditional Flatten + Dense layers. 
This is particularly beneficial for the Custom CNN, which otherwise has a higher risk of overfitting with fewer data.

Computationally, the Custom CNN trained faster ({custom_cnn_training_time:.2f}s) than the TL model ({tl_training_time:.2f}s). 
However, the TL model, despite having significantly more total parameters ({total_parameters:,}), 
had far fewer trainable parameters ({trainable_parameters:,}) compared to the Custom CNN's {custom_cnn.count_params():,}. 
This indicates that fine-tuning a pre-trained model can be computationally efficient for training, 
even if the overall model size is larger.

Convergence behavior also differed; the TL model achieved stable high accuracy much faster, 
with its validation loss decreasing rapidly and consistently. The Custom CNN showed slower and more fluctuating convergence. 
In essence, for image classification tasks with limited datasets, transfer learning is almost always the preferred approach, 
offering better performance, faster convergence, and reduced risk of overfitting due to its pre-trained knowledge and frozen layers.
"""


# REQUIRED: Print analysis with word count
print("ANALYSIS")
print(analysis_text)
print(f"Analysis word count: {len(analysis_text.split())} words")
if len(analysis_text.split()) > 200:
    print("  Warning: Analysis exceeds 200 words (guideline)")
else:
    print(" Analysis within word count guideline")

### 4.2 Visual Comparison

In [ ]:
def get_assignment_results():
    """
    Generate complete assignment results in required format
    """
    
    framework_used = "keras"
    
    results = {
        # Dataset Information
        'dataset_name': dataset_name,
        'dataset_source': dataset_source,
        'n_samples': int(n_samples),
        'n_classes': int(n_classes),
        'samples_per_class': samples_per_class,
        'image_shape': image_shape, 
        'problem_type': problem_type,
        'primary_metric': primary_metric,
        'metric_justification': metric_justification,
        'train_samples': int(final_train_samples),
        'test_samples': int(final_test_samples),
        'train_test_ratio': train_test_ratio,
        
        # Custom CNN Results
        'custom_cnn': {
            'framework': framework_used,
            'architecture': {
                'conv_layers': int(3), 
                'pooling_layers': int(3),
                'has_global_average_pooling': True,
                'output_layer': 'softmax',
                'total_parameters': int(custom_cnn.count_params())
            },
            'training_config': {
                'learning_rate': float(0.001),
                'n_epochs': int(EPOCHS),
                'batch_size': int(BATCH_SIZE),
                'optimizer': 'Adam',
                'loss_function': 'categorical_crossentropy'
            },
            'initial_loss': float(custom_cnn_initial_loss),
            'final_loss': float(custom_cnn_final_loss),
            'training_time_seconds': float(custom_cnn_training_time),
            'accuracy': float(custom_cnn_accuracy),
            'precision': float(custom_cnn_precision),
            'recall': float(custom_cnn_recall),
            'f1_score': float(custom_cnn_f1)
        },
        
        # Transfer Learning Results
        'transfer_learning': {
            'framework': framework_used,
            'base_model': pretrained_model_name,
            'frozen_layers': int(frozen_layers),
            'trainable_layers': int(trainable_layers),
            'has_global_average_pooling': True,
            'total_parameters': int(total_parameters),
            'trainable_parameters': int(trainable_parameters),
            'training_config': {
                'learning_rate': float(tl_learning_rate),
                'n_epochs': int(tl_epochs),
                'batch_size': int(tl_batch_size),
                'optimizer': tl_optimizer,
                'loss_function': 'categorical_crossentropy'
            },
            'initial_loss': float(tl_initial_loss),
            'final_loss': float(tl_initial_loss),
            'training_time_seconds': float(tl_training_time),
            'accuracy': float(tl_accuracy),
            'precision': float(tl_precision),
            'recall': float(tl_recall),
            'f1_score': float(tl_f1)
        },
        
        # Analysis
        'analysis': analysis_text,
        'analysis_word_count': int(len(analysis_text.split())),
        
        # Training Success Indicators
        'custom_cnn_loss_decreased': bool(custom_cnn_final_loss < custom_cnn_initial_loss),
        'transfer_learning_loss_decreased': bool(tl_final_loss < tl_initial_loss),
    }
    
    return results

# Generate and print results
try:
    assignment_results = get_assignment_results() 
    print("ASSIGNMENT RESULTS SUMMARY")
    print(json.dumps(assignment_results, indent=2))
    
except Exception as e:
    print(f"\n  ERROR generating results: {str(e)}")
    print("Please ensure all variables are properly defined and assigned.")   


import platform
import sys
from datetime import datetime

print("\nENVIRONMENT INFORMATION")
print("\n  REQUIRED: Add screenshot of your Google Colab/BITS Virtual Lab")
print("showing your account details in the cell below this one.")

print(f"Python Version: {sys.version}")
print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print(f"Operating System: {platform.system()} {platform.release()}")
print(f"Processor: {platform.processor()}")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if tf.config.list_physical_devices('GPU'):
    print("GPU: Available")
    for gpu in tf.config.list_physical_devices('GPU'):
        print(f"  - {gpu}")
else:
    print("GPU: Not Available, using CPU")